In [3]:
import pandas as pd

data = pd.read_csv(r'C:\Users\Caio\Desktop\projects\Python\Payments\transactions.csv', sep=';', header=0)

# 1. Transação de alto valor
mean_amount = data['transaction_amount'].mean()
std_amount = data['transaction_amount'].std()
threshold = mean_amount + (2 * std_amount)
high_value_transactions = data[data['transaction_amount'] > threshold]
num_high_value_transactions = len(high_value_transactions)

# 2. Histórico de chargebacks
chargeback_transactions = data[data['has_cbk'] == True]
num_chargeback_transactions = len(chargeback_transactions)

# 3. Compras com mesmo card_number em intervalo de 2 horas e valor acima de R$ 1000
data['transaction_date'] = pd.to_datetime(data['transaction_date'])
data['hour_of_day'] = data['transaction_date'].dt.hour

# 4. Transações repetidas em um intervalo de 1 hora e a soma das transações seja maior que R$ 1000,00
data['count_transactions_1h'] = data.groupby(['card_number', pd.Grouper(key='transaction_date', freq='1H')])['transaction_id'].transform('count')
data['sum_transactions_1h'] = data.groupby(['card_number', pd.Grouper(key='transaction_date', freq='1H')])['transaction_amount'].transform('sum')
repeated_transactions_1h = data[(data['count_transactions_1h'] > 1) & (data['sum_transactions_1h'] > 1000)]

# 5. Transações com mais de 3 transaction_id do mesmo card_number em um intervalo de 1 hora
data['count_transaction_id_1h'] = data.groupby(['card_number', pd.Grouper(key='transaction_date', freq='1H')])['transaction_id'].transform('nunique')
multiple_transaction_id_1h = data[data['count_transaction_id_1h'] > 3]

# 6. Obter as transações que não se enquadram em nenhuma das condições
not_matching_transactions = data[~data.index.isin(high_value_transactions.index) &
                                ~data.index.isin(chargeback_transactions.index) &
                                ~data.index.isin(repeated_transactions_1h.index) &
                                ~data.index.isin(multiple_transaction_id_1h.index)]

num_not_matching_transactions = len(not_matching_transactions)

# Impressão dos resultados
print("Transações de alto valor:")
display(high_value_transactions)
print("\nQuantidade de transações de alto valor:", num_high_value_transactions)
print("\n")

print("Transações com histórico de chargebacks:")
display(chargeback_transactions)
print("\nQuantidade de transações com chargebacks:", num_chargeback_transactions)
print("\n")

print("Transações repetidas em um intervalo de 1 hora e a soma das transações seja maior que R$ 1000,00:")
display(repeated_transactions_1h)
print("\nQuantidade de transações repetidas em um intervalo de 1 hora e a soma das transações seja maior que R$ 1000,00:", len(repeated_transactions_1h))
print("\n")

print("Transações com mais de 3 transaction_id do mesmo card_number em um intervalo de 1 hora:")
display(multiple_transaction_id_1h)
print("\nQuantidade de transações com mais de 3 transaction_id do mesmo card_number em um intervalo de 1 hora:", len(multiple_transaction_id_1h))
print("\n")

print("Transações que passaram em todas as análises de risco acima: ")
display(not_matching_transactions)
print("\nQuantidade de transações que passaram em todas as análises de risco acima: ", len(not_matching_transactions))

Transações de alto valor:


,transaction_id,merchant_id,user_id,card_number,transaction_date,transaction_amount,device_id,has_cbk
3,21320401,68657,69758,464296******3991,2019-12-01T21:59:19.797129,2556.13,NaN,True
26,21320424,53816,5541,606282******3381,2019-12-01T19:31:20.047571,2597.51,656429.0,True
32,21320430,56977,69758,464296******3991,2019-12-01T19:17:21.731168,2803.32,NaN,True
80,21320478,84546,55544,427167******310,2019-12-01T15:10:31.059068,2750.30,503714.0,False
86,21320484,84970,90182,464296******3991,2019-12-01T14:46:28.746288,3782.17,769545.0,True
...,...,...,...,...,...,...,...,...
3165,21323563,41354,19820,606282******6581,2019-11-02T16:33:21.333131,4031.00,NaN,True
3166,21323564,62052,69750,459383******9805,2019-11-02T16:08:15.927284,3084.48,NaN,False
3167,21323565,12729,73391,512363******3915,2019-11-02T15:35:54.844270,4054.06,NaN,False
3173,21323571,32186,72424,513557******2504,2019-11-02T12:09:00.766011,4044.35,NaN,False



Quantidade de transações de alto valor: 205


Transações com histórico de chargebacks:


,transaction_id,merchant_id,user_id,card_number,transaction_date,transaction_amount,device_id,has_cbk
1,21320399,92895,2708,444456******4210,2019-12-01T22:45:37.873639,734.87,497105.0,True
3,21320401,68657,69758,464296******3991,2019-12-01T21:59:19.797129,2556.13,NaN,True
7,21320405,56107,81152,650516******9201,2019-12-01T21:24:05.608374,188.68,486.0,True
8,21320406,56107,81152,650516******9201,2019-12-01T21:13:21.529999,352.77,486.0,True
9,21320407,56107,81152,650516******9201,2019-12-01T21:04:55.066909,345.68,486.0,True
...,...,...,...,...,...,...,...,...
3157,21323555,41354,19820,606282******6581,2019-11-02T18:04:22.088172,4028.55,NaN,True
3165,21323563,41354,19820,606282******6581,2019-11-02T16:33:21.333131,4031.00,NaN,True
3178,21323576,16266,96495,522840******2045,2019-11-01T22:10:49.517024,762.18,NaN,True
3180,21323578,16266,96495,552305******4782,2019-11-01T20:54:04.014855,2936.66,NaN,True



Quantidade de transações com chargebacks: 391


Transações repetidas em um intervalo de 1 hora e a soma das transações seja maior que R$ 1000,00:


...


Quantidade de transações repetidas em um intervalo de 1 hora e a soma das transações seja maior que R$ 1000,00: 121


Transações com mais de 3 transaction_id do mesmo card_number em um intervalo de 1 hora:


,transaction_id,merchant_id,user_id,card_number,transaction_date,transaction_amount,device_id,has_cbk,hour_of_day,count_transactions_1h,sum_transactions_1h,count_transaction_id_1h
312,21320710,44927,21768,530034******3859,2019-11-30 15:54:15.929622,987.13,357277.0,True,15,4,3969.76,4
323,21320721,44927,21768,530034******3859,2019-11-30 15:22:22.764974,1018.48,357277.0,True,15,4,3969.76,4
328,21320726,44927,21768,530034******3859,2019-11-30 15:08:59.460635,1000.26,357277.0,True,15,4,3969.76,4
331,21320729,44927,21768,530034******3859,2019-11-30 15:02:01.356559,963.89,357277.0,True,15,4,3969.76,4
2927,21323325,77130,75710,554482******7640,2019-11-08 23:18:28.634342,254.25,NaN,True,23,7,2829.78,7
2928,21323326,77130,75710,554482******7640,2019-11-08 23:17:08.619645,473.44,NaN,True,23,7,2829.78,7
2929,21323327,77130,75710,554482******7640,2019-11-08 23:15:05.322183,599.13,NaN,True,23,7,2829.78,7
2930,21323328,77130,75710,554482******7640,2019-11-08 23:14:35.977303,386.82,NaN,True,23,7,2829.78,7
2931,21323329,77130,75710,554482******7640,2019-11-08 23:12:00.465991,254.37,NaN,True,23,7,2829.78,7
2932,21323330,77130,75710,554482******7640,2019-11-08 23:11:18.971808,320.96,NaN,True,23,7,2829.78,7



Quantidade de transações com mais de 3 transaction_id do mesmo card_number em um intervalo de 1 hora: 15


Transações que passaram em todas as análises de risco acima: 


,transaction_id,merchant_id,user_id,card_number,transaction_date,transaction_amount,device_id,has_cbk,hour_of_day,count_transactions_1h,sum_transactions_1h,count_transaction_id_1h
0,21320398,29744,97051,434505******9116,2019-12-01 23:16:32.812632,374.56,285475.0,False,23,1,374.56,1
2,21320400,47759,14777,425850******7024,2019-12-01 22:22:43.021495,760.36,NaN,False,22,1,760.36,1
4,21320402,54075,64367,650487******6116,2019-12-01 21:30:53.347051,55.36,860232.0,False,21,1,55.36,1
5,21320403,59566,40759,516292******8220,2019-12-01 21:25:53.374213,60.49,192705.0,False,21,1,60.49,1
6,21320404,20917,25661,650485******9310,2019-12-01 21:25:19.532243,318.37,760932.0,False,21,1,318.37,1
...,...,...,...,...,...,...,...,...,...,...,...,...
3194,21323592,50493,49581,650486******4139,2019-11-01 13:05:34.054967,744.15,NaN,False,13,1,744.15,1
3195,21323593,9603,59275,528052******3611,2019-11-01 11:47:02.404963,1403.67,NaN,False,11,1,1403.67,1
3196,21323594,57997,84486,522688******9874,2019-11-01 10:23:50.555604,1.55,NaN,False,10,1,1.55,1
3197,21323595,35930,7,544315******7773,2019-11-01 01:29:45.799767,359.68,NaN,False,1,1,359.68,1



Quantidade de transações que passaram em todas as análises de risco acima:  2656
